# Advent of Code 2021 - Day 13

In [1]:
typealias Dot = Pair<Int, Int>

data class Instruction(val direction: Direction, val across: Int)

enum class Direction(val id: String) { 
    UP("y"),
    LEFT("x"),
    ;
    companion object {
        private val idMap: Map<String, Direction> = Direction.values().associateBy(Direction::id)
        fun fromId(id: String): Direction = idMap[id]!!
    }
}

data class Input(val dots: Set<Dot>, val instructions: List<Instruction>)

In [2]:
import java.io.File
import java.util.Scanner
import kotlin.sequences.generateSequence

val input: Input = Scanner(File("Day13.input.txt"))
    .apply { useDelimiter("\n\n") }
    .use { scanner ->
        generateSequence { if(scanner.hasNext()) scanner.next() else null }
            .toList()
            .let {
                val dotsRegex = Regex("""(\d+),(\d+)""")
                val instructionRegex = Regex("""(x|y)=(\d+)""")
                Input(
                    dots = it.first().lineSequence()
                        .map { dotsRegex.find(it)!!.destructured }
                        .map { (x, y) -> x.toInt() to y.toInt() }
                        .toSet(),
                    instructions = it.last().lineSequence()
                        .map { instructionRegex.find(it)!!.destructured }
                        .map { (direction, at) -> Instruction(Direction.fromId(direction), at.toInt())}
                        .toList()
                )
            }
    }

## Part 1

The set of `Input::dots` corresponds to locations on a grid. Fold the grid according to the first `Input::instructions` where the `Instruction::across` represents the horizontal or vertical line to mirror the dot across. How many dots are visible after the first fold?

In [3]:
fun Dot.fold(inst: Instruction): Dot =
    when (inst.direction) {
        Direction.UP -> if (second < inst.across) this else first to (second - ((second - inst.across) * 2))
        Direction.LEFT -> if (first < inst.across) this else (first - ((first - inst.across) * 2)) to second
    }

fun Set<Dot>.fold(inst: Instruction): Set<Dot> = map { it.fold(inst) }.toSet()

input.dots.fold(input.instructions[0]).size

664

### Notes

The trick here is to figure out how to "fold" or "mirror" an individual dot across the line. To fold it across a horizontal line, subtract the y position of the dot by 2 times the difference between the dot and the line. The rule is similar for folding left except with the x position. 

## Part 2

Fold the `Input::dots` through all of the `Input::instructions`. Find the final 8 capital letter code.

In [4]:
fun Set<Dot>.display(mark: String = "#", empty: String = "."): Unit {
    (0..maxOf { it.second }).forEach { y ->
        (0..maxOf { it.first }).forEach { x ->
            print(if ((x to y) in this) mark else empty)
        }
        println()
    }
}

input.instructions.fold(input.dots) { previous, instruction -> previous.fold(instruction) }
    .display("🇹🇭", "🏴")

🇹🇭🇹🇭🇹🇭🇹🇭🏴🇹🇭🇹🇭🇹🇭🇹🇭🏴🏴🏴🇹🇭🇹🇭🏴🇹🇭🏴🏴🇹🇭🏴🇹🇭🇹🇭🇹🇭🇹🇭🏴🇹🇭🏴🏴🏴🏴🇹🇭🇹🇭🇹🇭🏴🏴🇹🇭🏴🏴🏴
🇹🇭🏴🏴🏴🏴🇹🇭🏴🏴🏴🏴🏴🏴🏴🇹🇭🏴🇹🇭🏴🇹🇭🏴🏴🏴🏴🏴🇹🇭🏴🇹🇭🏴🏴🏴🏴🇹🇭🏴🏴🇹🇭🏴🇹🇭🏴🏴🏴
🇹🇭🇹🇭🇹🇭🏴🏴🇹🇭🇹🇭🇹🇭🏴🏴🏴🏴🏴🇹🇭🏴🇹🇭🇹🇭🏴🏴🏴🏴🏴🇹🇭🏴🏴🇹🇭🏴🏴🏴🏴🇹🇭🇹🇭🇹🇭🏴🏴🇹🇭🏴🏴🏴
🇹🇭🏴🏴🏴🏴🇹🇭🏴🏴🏴🏴🏴🏴🏴🇹🇭🏴🇹🇭🏴🇹🇭🏴🏴🏴🇹🇭🏴🏴🏴🇹🇭🏴🏴🏴🏴🇹🇭🏴🏴🇹🇭🏴🇹🇭🏴🏴🏴
🇹🇭🏴🏴🏴🏴🇹🇭🏴🏴🏴🏴🇹🇭🏴🏴🇹🇭🏴🇹🇭🏴🇹🇭🏴🏴🇹🇭🏴🏴🏴🏴🇹🇭🏴🏴🏴🏴🇹🇭🏴🏴🇹🇭🏴🇹🇭🏴🏴🏴
🇹🇭🇹🇭🇹🇭🇹🇭🏴🇹🇭🏴🏴🏴🏴🏴🇹🇭🇹🇭🏴🏴🇹🇭🏴🏴🇹🇭🏴🇹🇭🇹🇭🇹🇭🇹🇭🏴🇹🇭🇹🇭🇹🇭🇹🇭🏴🇹🇭🇹🇭🇹🇭🏴🏴🇹🇭🇹🇭🇹🇭🇹🇭


### Notes

This one was really fun / satisfying to solve. The folding logic still works from part 1, so now we just need to figure out how to display the dots. To do so, we can just iteratively go through a grid and print a unique symbol every time the index was a dot.